### Imports

In [14]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
from datetime import date, datetime
import ast
import os
import matplotlib.pyplot as plt
import math
import json

### Define DA model with Flexibility Options and Storage Support

In [15]:
class DAFOModel:
    def __init__(self, num_periods=24, num_scenarios=5, num_generators=0, num_tiers=4, num_storage=0):
        self.num_periods = num_periods
        self.num_scenarios = num_scenarios
        self.num_generators = num_generators
        self.num_tiers = num_tiers
        self.num_storage = num_storage
        
        self.model = pyo.AbstractModel()
        self._define_sets()
        self._define_parameters()
        self._define_variables()
        self._define_objective()
        self._define_constraints()
    
    def _define_sets(self):
        self.model.T = pyo.RangeSet(1, self.num_periods)    # Set of time periods
        self.model.S = pyo.RangeSet(1, self.num_scenarios)  # Set of scenarios
        self.model.R = pyo.RangeSet(1, self.num_tiers)      # Set of tiers
        
        # Generator set
        if self.num_generators > 0:
            self.model.G = pyo.RangeSet(1, self.num_generators)
        else:
            self.model.G = pyo.Set()
        
        # Storage set
        if self.num_storage > 0:
            self.model.B = pyo.RangeSet(1, self.num_storage)
        else:
            self.model.B = pyo.Set()
    
    def _define_parameters(self):
        # General Parameters
        self.model.VC = pyo.Param(self.model.G)           # Variable cost
        self.model.VCUP = pyo.Param(self.model.G)         # Variable cost up
        self.model.VCDN = pyo.Param(self.model.G)         # Variable cost down
        self.model.CAP = pyo.Param(self.model.G)          # Capacity
        self.model.REDA = pyo.Param(self.model.T)         # Maximum DA RE for each hour
        self.model.DEMAND = pyo.Param(self.model.T)       # Electricity demand per hour
        self.model.D1 = pyo.Param(within=pyo.NonNegativeIntegers)    # Linear cost coefficient for demand slack
        self.model.D2 = pyo.Param(within=pyo.NonNegativeIntegers)    # Quadratic cost coefficient for demand slack

        # Parameters specific to the FO
        self.model.RR = pyo.Param(self.model.G)                      # Ramp rate
        self.model.RE = pyo.Param(self.model.S, self.model.T)        # Renewable generation at each scenario and time
        self.model.PEN = pyo.Param(within=pyo.NonNegativeIntegers)   # Penalty for inadequate flexibility up
        self.model.PENDN = pyo.Param(within=pyo.NonNegativeIntegers) # Penalty for inadequate flexibility down
        self.model.smallM = pyo.Param(within=pyo.NonNegativeReals)   # Parameter for alternative optima
        self.model.probTU = pyo.Param(self.model.R)                  # Probability of exercise FO up
        self.model.probTD = pyo.Param(self.model.R)                  # Probability of exercise FO down

        # Storage Parameters
        self.model.E_MAX = pyo.Param(self.model.B)        # Maximum energy capacity
        self.model.P_MAX = pyo.Param(self.model.B)        # Maximum power capacity
        self.model.ETA_CH = pyo.Param(self.model.B)       # Charging efficiency
        self.model.ETA_DCH = pyo.Param(self.model.B)      # Discharging efficiency
        self.model.E0 = pyo.Param(self.model.B)           # Initial state of charge
        self.model.E_FINAL = pyo.Param(self.model.B)      # Required final state of charge
        self.model.STORAGE_COST = pyo.Param(self.model.B) # Storage operating cost per MWh
    
    def _define_variables(self):
        # Energy and reserve variables
        self.model.d = pyo.Var(self.model.T, domain=pyo.NonNegativeReals)      # Demand slack
        self.model.rgDA = pyo.Var(self.model.T, domain=pyo.NonNegativeReals)   # DA renewables schedule
        self.model.du = pyo.Var(self.model.S, self.model.T)                    # Demand uncertainty
        
        # Variables dependent on generator set
        if len(self.model.G) > 0 or isinstance(self.model.G, pyo.RangeSet):
            self.model.xDA = pyo.Var(self.model.G, self.model.T, domain=pyo.NonNegativeReals)  # DA energy schedule
            # generator FO Variables
            self.model.hsu = pyo.Var(self.model.R, self.model.G, self.model.T, domain=pyo.NonNegativeReals)  # Supply FO up
            self.model.hsd = pyo.Var(self.model.R, self.model.G, self.model.T, domain=pyo.NonNegativeReals)  # Supply FO down
            
        # FO Variables independent of generators and storage
        self.model.hdu = pyo.Var(self.model.R, self.model.T, domain=pyo.NonNegativeReals)     # Demand FO up
        self.model.hdd = pyo.Var(self.model.R, self.model.T, domain=pyo.NonNegativeReals)     # Demand FO down
        self.model.sdu = pyo.Var(self.model.R, self.model.T, domain=pyo.NonNegativeReals)     # Self-supply FO up
        self.model.sdd = pyo.Var(self.model.R, self.model.T, domain=pyo.NonNegativeReals)     # Self-supply FO down
        self.model.y = pyo.Var(self.model.S, self.model.T, domain=pyo.NonNegativeReals)       # Auxiliary variable

        # Variables dependent on storage set
        if len(self.model.B) > 0 or isinstance(self.model.B, pyo.RangeSet):
            # Storage Variables
            self.model.e = pyo.Var(self.model.B, self.model.T, domain=pyo.NonNegativeReals)     # Energy level
            self.model.p_ch = pyo.Var(self.model.B, self.model.T, domain=pyo.NonNegativeReals)  # Charging power
            self.model.p_dch = pyo.Var(self.model.B, self.model.T, domain=pyo.NonNegativeReals) # Discharging power
            # Storage FO Variables
            self.model.bsu = pyo.Var(self.model.R, self.model.B, self.model.T, domain=pyo.NonNegativeReals)  # Storage FO up
            self.model.bsd = pyo.Var(self.model.R, self.model.B, self.model.T, domain=pyo.NonNegativeReals)  # Storage FO down
    
    def _define_objective(self):
        # Objective function
        def obj_expression(m):
            objective_terms = []
            
            # objective_terms.append(sum(m.VC.get(g, 0) * m.xDA[g,t] 
            #                         for g in m.G for t in m.T 
            #                         if g in m.VC))
        
            
            # Energy costs - generators
            if hasattr(m, 'xDA') and len(m.G) > 0:
                objective_terms.append(sum(m.VC[g] * m.xDA[g,t] for g in m.G for t in m.T))
            
            # Flexibility costs - generators
            if hasattr(m, 'hsu') and len(m.G) > 0:
                objective_terms.append(sum(m.probTU[r] * m.VCUP[g] * m.hsu[r,g,t] for g in m.G for r in m.R for t in m.T))
                objective_terms.append(-sum(m.probTD[r] * m.VCDN[g] * m.hsd[r,g,t] for g in m.G for r in m.R for t in m.T))
            
            # Self-supply flexibility costs
            objective_terms.append(sum(m.probTU[r] * m.PEN * m.sdu[r,t] for r in m.R for t in m.T))
            objective_terms.append(-sum(m.probTD[r] * m.PENDN * m.sdd[r,t] for r in m.R for t in m.T))
            
            # Storage related costs
            if hasattr(m, 'bsu') and len(m.B) > 0:
                # Storage flexibility costs
                objective_terms.append(sum(m.probTU[r] * m.STORAGE_COST[b] * m.bsu[r,b,t] for b in m.B for r in m.R for t in m.T))
                objective_terms.append(-sum(m.probTD[r] * m.STORAGE_COST[b] * m.bsd[r,b,t] for b in m.B for r in m.R for t in m.T))
                # Storage operation costs
                objective_terms.append(sum(m.STORAGE_COST[b] * (m.p_ch[b,t] + m.p_dch[b,t]) for b in m.B for t in m.T))
            
            # Auxiliary costs
            objective_terms.append(sum(m.y[s,t] for s in m.S for t in m.T) * m.smallM)
            
            # Demand response costs
            objective_terms.append(sum(0.2 * m.D1 * (m.d[t] + m.du[s,t]) for s in m.S for t in m.T))
            objective_terms.append(0.2 * m.D2 * sum((m.d[t] + m.du[s,t]) * (m.d[t] + m.du[s,t]) for s in m.S for t in m.T))
            
            return sum(objective_terms)

        self.model.OBJ = pyo.Objective(rule=obj_expression)
    
    def _define_constraints(self):
        # Define constraints - Numbering of constraints follows paper
        # Energy balance for each hour
        def DA_energy_balance(model, t):
            gen_sum = 0
            storage_sum = 0
            
            if hasattr(model, 'xDA'):
                gen_sum = sum(model.xDA[g,t] for g in model.G) if len(model.G) > 0 else 0
                
            if hasattr(model, 'p_dch') and hasattr(model, 'p_ch'):
                storage_sum = sum(model.p_dch[b,t] - model.p_ch[b,t] for b in model.B) if len(model.B) > 0 else 0
            
            return (gen_sum + 
                    model.rgDA[t] + 
                    storage_sum + 
                    model.d[t] == model.DEMAND[t])
        
        self.model.Con3 = pyo.Constraint(self.model.T, rule=DA_energy_balance)

        # # Flexibility balance for each hour
        # def DA_flexup_balance(model, r, t):
        #     gen_flex_up = 0
        #     storage_flex_up = 0
            
        #     if hasattr(model, 'hsu'):
        #         gen_flex_up = sum(model.hsu[r,g,t] for g in model.G) if len(model.G) > 0 else 0
                
        #     if hasattr(model, 'bsu'):
        #         storage_flex_up = sum(model.bsu[r,b,t] for b in model.B) if len(model.B) > 0 else 0
            
        #     return (gen_flex_up + storage_flex_up == model.hdu[r,t])
        
        # self.model.Con4UP = pyo.Constraint(self.model.R, self.model.T, rule=DA_flexup_balance)

        # def DA_flexdn_balance(model, r, t):
        #     gen_flex_down = 0
        #     storage_flex_down = 0
            
        #     if hasattr(model, 'hsd'):
        #         gen_flex_down = sum(model.hsd[r,g,t] for g in model.G) if len(model.G) > 0 else 0
                
        #     if hasattr(model, 'bsd'):
        #         storage_flex_down = sum(model.bsd[r,b,t] for b in model.B) if len(model.B) > 0 else 0
            
        #     return (gen_flex_down + storage_flex_down == model.hdd[r,t])
        
        # self.model.Con4DN = pyo.Constraint(self.model.R, self.model.T, rule=DA_flexdn_balance)

        # # Flexibility demand for each scenario and hour
        # def DA_flex_demand(model, s, t):
        #     return (-model.du[s,t] + 
        #             sum(model.hdd[r,t] + model.sdd[r,t] for r in model.R if r <= s-1) -
        #             sum(model.hdu[r,t] + model.sdu[r,t] for r in model.R if r >= s) == 
        #             model.RE[s,t] - model.rgDA[t])
        
        # self.model.Con6 = pyo.Constraint(self.model.S, self.model.T, rule=DA_flex_demand)

        # # Add generator constraints only if generators exist
        # if hasattr(self.model, 'xDA') and (len(self.model.G) > 0 or isinstance(self.model.G, pyo.RangeSet) or self.num_generators is None):
        #     # Inter-temporal constraints
        #     def ramp_rate_up(model, g, t):
        #         if t == 1:
        #             return pyo.Constraint.Skip
        #         return model.xDA[g,t] - model.xDA[g,t-1] <= model.RR[g]
            
        #     self.model.ramp_up = pyo.Constraint(self.model.G, self.model.T, rule=ramp_rate_up)

        #     def ramp_rate_down(model, g, t):
        #         if t == 1:
        #             return pyo.Constraint.Skip
        #         return model.xDA[g,t-1] - model.xDA[g,t] <= model.RR[g]
            
        #     self.model.ramp_down = pyo.Constraint(self.model.G, self.model.T, rule=ramp_rate_down)

        #     # Generation limits without commitment status
        #     def generation_limits(model, g, t):
        #         return model.xDA[g,t] <= model.CAP[g]
            
        #     self.model.generation_limits = pyo.Constraint(self.model.G, self.model.T, rule=generation_limits)

        # # Add storage constraints only if storage exists
        # if hasattr(self.model, 'e') and (len(self.model.B) > 0 or isinstance(self.model.B, pyo.RangeSet) or self.num_storage is None):
        #     # Storage energy balance
        #     def storage_balance(model, b, t):
        #         if t == 1:
        #             return (model.e[b,t] == model.E0[b] + 
        #                     model.ETA_CH[b] * model.p_ch[b,t] - 
        #                     (1/model.ETA_DCH[b]) * model.p_dch[b,t])
        #         else:
        #             return (model.e[b,t] == model.e[b,t-1] + 
        #                     model.ETA_CH[b] * model.p_ch[b,t] - 
        #                     (1/model.ETA_DCH[b]) * model.p_dch[b,t])

        # Record duals for market analysis
        self.model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
        
    def create_instance(self, data):
        return self.model.create_instance(data)

### RT Simulation Model with Storage Integration

In [16]:
class RTSimModel:
    def __init__(self, num_periods=24, num_scenarios=5, num_generators=None, num_storage=None):
        self.num_periods = num_periods
        self.num_scenarios = num_scenarios
        self.num_generators = num_generators
        self.num_storage = num_storage
        
        self.model = pyo.AbstractModel()
        self._define_sets()
        self._define_parameters()
        self._define_variables()
        self._define_objective()
        self._define_constraints()
    
    def _define_sets(self):
        # Sets
        self.model.T = pyo.RangeSet(1, self.num_periods)    # Set of time periods
        self.model.S = pyo.RangeSet(1, self.num_scenarios)  # Set of scenarios
        
        # Generator set - can be determined from data
        if self.num_generators is not None and self.num_generators > 0:
            self.model.G = pyo.RangeSet(1, self.num_generators)
        else:
            self.model.G = pyo.Set()  # Will be populated when data is loaded or can be empty
        
        # Storage set - can be determined from data
        if self.num_storage is not None and self.num_storage > 0:
            self.model.B = pyo.RangeSet(1, self.num_storage)
        else:
            self.model.B = pyo.Set()  # Will be populated when data is loaded or can be empty
    
    def _define_parameters(self):
        # Original Parameters
        self.model.VC = pyo.Param(self.model.G)           # Variable cost
        self.model.VCUP = pyo.Param(self.model.G)         # Variable cost up
        self.model.VCDN = pyo.Param(self.model.G)         # Variable cost down
        self.model.CAP = pyo.Param(self.model.G)          # Generator capacity
        self.model.RR = pyo.Param(self.model.G)           # Ramp rate

        self.model.prob = pyo.Param(self.model.S)         # Scenario probability
        self.model.RE = pyo.Param(self.model.S, self.model.T)  # Renewable generation by scenario and time
        self.model.DEMAND = pyo.Param(self.model.T)       # Hourly demand
        self.model.D1 = pyo.Param(within=pyo.NonNegativeIntegers)  # Linear demand cost coefficient
        self.model.D2 = pyo.Param(within=pyo.NonNegativeIntegers)  # Quadratic demand cost coefficient
        self.model.xDA = pyo.Param(self.model.G, self.model.T) # DA schedule by generator and time
        self.model.REDA = pyo.Param(self.model.T)         # DA renewable schedule by time
        self.model.PEN = pyo.Param(within=pyo.NonNegativeIntegers)   # Upward penalty
        self.model.PENDN = pyo.Param()                    # Downward penalty
        self.model.DAdr = pyo.Param(self.model.T)         # DA demand response by time

        # Storage Parameters
        self.model.E_MAX = pyo.Param(self.model.B)        # Maximum energy capacity
        self.model.P_MAX = pyo.Param(self.model.B)        # Maximum power capacity
        self.model.ETA_CH = pyo.Param(self.model.B)       # Charging efficiency
        self.model.ETA_DCH = pyo.Param(self.model.B)      # Discharging efficiency
        self.model.E0 = pyo.Param(self.model.B)           # Initial state of charge
        self.model.STORAGE_COST = pyo.Param(self.model.B)  # Operating cost per MWh of throughput
        self.model.E_FINAL = pyo.Param(self.model.B)      # Required final state of charge
        
        # Storage DA parameters
        self.model.e_DA = pyo.Param(self.model.B, self.model.T)  # DA energy level
        self.model.p_ch_DA = pyo.Param(self.model.B, self.model.T)  # DA charging
        self.model.p_dch_DA = pyo.Param(self.model.B, self.model.T)  # DA discharging
    
    def _define_variables(self):
        # Variables that depend on generators
        if len(self.model.G) > 0 or isinstance(self.model.G, pyo.RangeSet) or self.num_generators is None:
            # RT adjustment variables for each scenario and time
            self.model.xup = pyo.Var(self.model.S, self.model.G, self.model.T, domain=pyo.NonNegativeReals)  # Generator up adjustment
            self.model.xdn = pyo.Var(self.model.S, self.model.G, self.model.T, domain=pyo.NonNegativeReals)  # Generator down adjustment
        
        # Variables independent of generators and storage
        self.model.d = pyo.Var(self.model.S, self.model.T)     # RT demand response
        self.model.rgup = pyo.Var(self.model.S, self.model.T, domain=pyo.NonNegativeReals)  # RE up adjustment
        self.model.rgdn = pyo.Var(self.model.S, self.model.T, domain=pyo.NonNegativeReals)  # RE down adjustment
        self.model.sdup = pyo.Var(self.model.S, self.model.T, domain=pyo.NonNegativeReals)  # Shortage
        self.model.sddn = pyo.Var(self.model.S, self.model.T, domain=pyo.NonNegativeReals)  # Surplus

        # Variables that depend on storage
        if len(self.model.B) > 0 or isinstance(self.model.B, pyo.RangeSet) or self.num_storage is None:
            # Storage Variables
            self.model.e = pyo.Var(self.model.S, self.model.B, self.model.T, domain=pyo.NonNegativeReals)     # Energy level
            self.model.p_ch = pyo.Var(self.model.S, self.model.B, self.model.T, domain=pyo.NonNegativeReals)  # Charging power
            self.model.p_dch = pyo.Var(self.model.S, self.model.B, self.model.T, domain=pyo.NonNegativeReals) # Discharging power
            self.model.b_up = pyo.Var(self.model.S, self.model.B, self.model.T, domain=pyo.NonNegativeReals)  # Storage up adjustment
            self.model.b_dn = pyo.Var(self.model.S, self.model.B, self.model.T, domain=pyo.NonNegativeReals)  # Storage down adjustment
    
    def _define_objective(self):
        # Objective Function
        def obj_expression(m):
            objective_terms = []
            
            for s in m.S:
                scenario_terms = []
                
                # Generator adjustment costs - only if generators exist
                if hasattr(m, 'xup') and len(m.G) > 0:
                    scenario_terms.append(
                        sum(m.VCUP[g] * m.xup[s,g,t] - m.VCDN[g] * m.xdn[s,g,t] for g in m.G for t in m.T)
                    )
                
                # Shortage/surplus penalties
                scenario_terms.append(m.PENDN * sum(m.sdup[s,t] for t in m.T))
                scenario_terms.append(m.PEN * sum(m.sddn[s,t] for t in m.T))
                
                # Demand response costs
                scenario_terms.append(
                    sum((m.D1 * (m.DAdr[t] + m.d[s,t]) + 
                         m.D2 * (m.DAdr[t] + m.d[s,t]) * (m.DAdr[t] + m.d[s,t])) for t in m.T)
                )
                scenario_terms.append(
                    -sum((m.D1 * m.DAdr[t] + m.D2 * m.DAdr[t] * m.DAdr[t]) for t in m.T)
                )
                
                # Storage operating costs - only if storage exists
                if hasattr(m, 'p_ch') and len(m.B) > 0:
                    scenario_terms.append(
                        sum(m.STORAGE_COST[b] * (m.p_ch[s,b,t] + m.p_dch[s,b,t]) for b in m.B for t in m.T)
                    )
                
                # Add the weighted scenario terms to the overall objective
                objective_terms.append(m.prob[s] * sum(scenario_terms))
            
            return sum(objective_terms)

        self.model.OBJ = pyo.Objective(rule=obj_expression)
    
    def _define_constraints(self):
        # RT energy balance for each scenario and time period
        def RT_energy_balance(model, s, t):
            gen_adjustment = 0
            storage_adjustment = 0
            
            # Generator adjustments - only if generators exist
            if hasattr(model, 'xup') and len(model.G) > 0:
                gen_adjustment = sum(model.xup[s,g,t] - model.xdn[s,g,t] for g in model.G)
            
            # Storage adjustments - only if storage exists
            if hasattr(model, 'p_ch') and len(model.B) > 0:
                storage_adjustment = sum(
                    model.p_dch[s,b,t] - model.p_ch[s,b,t] - 
                    model.p_dch_DA[b,t] + model.p_ch_DA[b,t] for b in model.B
                )
            
            return (gen_adjustment +
                    model.rgup[s,t] - model.rgdn[s,t] +
                    storage_adjustment +
                    model.d[s,t] == 0)
                    
        self.model.Con3 = pyo.Constraint(self.model.S, self.model.T, rule=RT_energy_balance)

        # RT renewable availability
        def RT_RE_availability(model, s, t):
            return (model.rgup[s,t] - model.rgdn[s,t] + model.sdup[s,t] - model.sddn[s,t] == 
                    model.RE[s,t] - model.REDA[t])
                    
        self.model.Con4 = pyo.Constraint(self.model.S, self.model.T, rule=RT_RE_availability)

        # Generator constraints - only if generators exist
        if hasattr(self.model, 'xup') and (len(self.model.G) > 0 or isinstance(self.model.G, pyo.RangeSet) or self.num_generators is None):
            # Generator ramping constraints
            def RT_ramp_up(model, s, g, t):
                return model.xup[s,g,t] <= model.RR[g]
                
            self.model.Con5up = pyo.Constraint(self.model.S, self.model.G, self.model.T, rule=RT_ramp_up)

            def RT_ramp_dn(model, s, g, t):
                return model.xdn[s,g,t] <= model.RR[g]
                
            self.model.Con5dn = pyo.Constraint(self.model.S, self.model.G, self.model.T, rule=RT_ramp_dn)

            # Generator capacity constraints
            def RT_capacity_cons(model, s, g, t):
                return model.xDA[g,t] + model.xup[s,g,t] <= model.CAP[g]
                
            self.model.Con6 = pyo.Constraint(self.model.S, self.model.G, self.model.T, rule=RT_capacity_cons)

            def RT_capacity_min(model, s, g, t):
                return model.xDA[g,t] - model.xdn[s,g,t] >= 0
                
            self.model.Con7 = pyo.Constraint(self.model.S, self.model.G, self.model.T, rule=RT_capacity_min)

        # Storage Constraints - only if storage exists
        if hasattr(self.model, 'e') and (len(self.model.B) > 0 or isinstance(self.model.B, pyo.RangeSet) or self.num_storage is None):
            # Storage energy balance
            def storage_balance(model, s, b, t):
                if t == 1:
                    return (model.e[s,b,t] == model.E0[b] + 
                            model.ETA_CH[b] * model.p_ch[s,b,t] - 
                            (1/model.ETA_DCH[b]) * model.p_dch[s,b,t])
                return (model.e[s,b,t] == model.e[s,b,t-1] + 
                        model.ETA_CH[b] * model.p_ch[s,b,t] - 
                        (1/model.ETA_DCH[b]) * model.p_dch[s,b,t])
                        
            self.model.storage_balance = pyo.Constraint(self.model.S, self.model.B, self.model.T, rule=storage_balance)

            # Storage capacity constraint
            def storage_capacity(model, s, b, t):
                return model.e[s,b,t] <= model.E_MAX[b]
                
            self.model.storage_capacity = pyo.Constraint(self.model.S, self.model.B, self.model.T, rule=storage_capacity)

            # Power limits
            def power_limits(model, s, b, t):
                return model.p_ch[s,b,t] + model.p_dch[s,b,t] <= model.P_MAX[b]
                
            self.model.power_limits = pyo.Constraint(self.model.S, self.model.B, self.model.T, rule=power_limits)
            
            # Storage adjustment limits
            def storage_adjustment_limits(model, s, b, t):
                return model.b_up[s,b,t] + model.b_dn[s,b,t] <= 0.5 * model.P_MAX[b]
                
            self.model.storage_adjustment_limits = pyo.Constraint(self.model.S, self.model.B, self.model.T, 
                                                                rule=storage_adjustment_limits)

            # Final state of charge requirement
            def final_soc(model, s, b):
                return model.e[s,b,self.num_periods] >= model.E_FINAL[b]
                
            self.model.final_soc = pyo.Constraint(self.model.S, self.model.B, rule=final_soc)

            # Storage ramping constraints
            def storage_ramp_rate(model, s, b, t):
                if t == 1:
                    return pyo.Constraint.Skip
                # This constraint with 'abs' needs to be reformulated for a solver
                # Here's a simple reformulation with two constraints
                return [
                    model.p_ch[s,b,t] - model.p_ch[s,b,t-1] <= 0.25 * model.P_MAX[b],
                    model.p_ch[s,b,t-1] - model.p_ch[s,b,t] <= 0.25 * model.P_MAX[b],
                    model.p_dch[s,b,t] - model.p_dch[s,b,t-1] <= 0.25 * model.P_MAX[b],
                    model.p_dch[s,b,t-1] - model.p_dch[s,b,t] <= 0.25 * model.P_MAX[b]
                ]
                        
            self.model.storage_ramp = pyo.Constraint(self.model.S, self.model.B, self.model.T, rule=storage_ramp_rate)

        # Record duals
        self.model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
        
    def create_instance(self, data):
        return self.model.create_instance(data)

Data Processing

In [37]:
class SystemDataProcessor:
    def __init__(self, gen_csv_path, storage_csv_path, demand_csv_path):
        self.gen_csv_path = gen_csv_path
        self.storage_csv_path = storage_csv_path
        self.demand_csv_path = demand_csv_path
        self.gen_data = None
        self.storage_data = None
        self.demand_data = None

    def load_data(self):
        try:
            self.gen_data = pd.read_csv(self.gen_csv_path).head(5)
            self.storage_data = pd.read_csv(self.storage_csv_path).head(5)
            print(f"Generator and Storage data loaded successfully. {len(self.gen_data)} generators and {len(self.storage_data)} storages found.")
            self.demand_data = pd.read_csv(self.demand_csv_path)
            print(f"Demand data loaded successfully. {len(self.demand_data)} periods found.")
            return True
        except Exception as e:
            print(f"Error loading data: {str(e)}")
            return False

    def process_gen_data(self):
        id_col = 'GEN UID'
        column_mapping = {
            # 'Ramp Rate MW/Min': '', 
            # 'Fuel Price $/MMBTU': '', 
            # 'VOM': '',
            'PMax MW': 'CAP',
            'Ramp Rate MW/Min': 'RR',
        }

        relevant_cols = list(column_mapping.keys())
        existing_cols = [col for col in relevant_cols if col in self.gen_data.columns]
        selected_cols = [id_col] + existing_cols
        gen_data_filtered = self.gen_data[selected_cols].copy()

        rename_dict = {col: column_mapping[col] for col in column_mapping if col in gen_data_filtered.columns}
        gen_data_filtered.rename(columns=rename_dict, inplace=True)

        original_ids = gen_data_filtered[id_col].tolist()
        gen_id_mapping = {original_id: i+1 for i, original_id in enumerate(original_ids)}
        gen_data_filtered[id_col] = gen_data_filtered[id_col].map(gen_id_mapping)
        
        gen_data_filtered.set_index(id_col, inplace=True)

        # if 'Fuel Price $/MMBTU' in gen_data_filtered.columns and 'HR_avg_0' in self.gen_data.columns:
        #     gen_data_filtered['VC'] = self.gen_data['Fuel Price $/MMBTU'] * self.gen_data['HR_avg_0'] / 1000.0
        #     if 'VOM' in gen_data_filtered.columns:
        #         gen_data_filtered['VC'] += gen_data_filtered['VOM']
        
        # TODO - CALCULATE VC, VCUP, VCDN
        gen_data_filtered['VC'] = 20
        gen_data_filtered['VCUP'] = 20
        gen_data_filtered['VCDN'] = 20

        params = ['CAP', 'RR','VC','VCUP','VCDN']
        gen_data_dict = {}

        for param in params:
            if param in gen_data_filtered.columns:
                param_dict = {}
                for gen_idx in gen_data_filtered.index:
                    param_dict[gen_idx] = gen_data_filtered.loc[gen_idx, param]
                gen_data_dict[param] = param_dict
            
        return gen_data_dict

    def process_storage_data(self):
        if self.storage_data is None or self.storage_data.empty:
            return {}
            
        try:
            id_col = 'GEN UID'

            column_mapping = {
                'Max Volume GWh': 'E_MAX',
                'Rating MVA': 'P_MAX',
                'Initial Volume GWh': 'E0',
                # 'Storage Roundtrip Efficiency': 'ETA'
            }

            relevant_cols = list(column_mapping.keys())
            existing_cols = [col for col in relevant_cols if col in self.storage_data.columns]
                
            selected_cols = [id_col] + existing_cols
            storage_data_filtered = self.storage_data[selected_cols].copy()

            rename_dict = {col: column_mapping[col] for col in column_mapping if col in storage_data_filtered.columns}
            storage_data_filtered.rename(columns=rename_dict, inplace=True)

            original_ids = storage_data_filtered[id_col].tolist()
            storage_id_mapping = {original_id: i+1 for i, original_id in enumerate(original_ids)}
            storage_data_filtered[id_col] = storage_data_filtered[id_col].map(storage_id_mapping)
        
            storage_data_filtered.set_index(id_col, inplace=True)

            storage_data_dict = {}

            if 'E_MAX' in storage_data_filtered.columns:
                storage_data_dict['E_MAX'] = {
                    storage_idx: float(storage_data_filtered.at[storage_idx, 'E_MAX']) * 1000.0  # GWh to MWh
                    for storage_idx in storage_data_filtered.index
                }
                
            if 'P_MAX' in storage_data_filtered.columns:
                storage_data_dict['P_MAX'] = {
                    storage_idx: float(storage_data_filtered.loc[storage_idx, 'P_MAX'])
                    for storage_idx in storage_data_filtered.index
                }
            
            # check charging and discharging efficiency
            storage_data_dict['ETA_CH'] = {
                storage_idx: 0.9    
                for storage_idx in storage_data_filtered.index
            }
            storage_data_dict['ETA_DCH'] = {
                storage_idx: 0.9
                for storage_idx in storage_data_filtered.index
            }
            storage_data_dict['STORAGE_COST'] = {
                storage_idx: 0.9
                for storage_idx in storage_data_filtered.index
            }
            storage_data_dict['E0'] = {
                storage_idx: 0.0
                for storage_idx in storage_data_filtered.index
            }
            storage_data_dict['E_FINAL'] = {
                storage_idx: 0.0
                for storage_idx in storage_data_filtered.index
            }

            # if 'E0' in storage_data_filtered.columns:
            #     storage_data_dict['E0'] = {
            #         storage_idx: float(storage_data_filtered.loc[old_idx, 'E0']) * 1000.0  # GWh to MWh
            #         for old_idx, storage_idx in storage_indices.items()
            #     }

            return storage_data_dict
            
        except Exception as e:
            print(f"Error processing storage data: {str(e)}")
            return {}
    
    def process_demand_data(self, num_periods=24):
        try:
            demand_data = self.demand_data.head(num_periods)
            demand_dict = demand_data.set_index(demand_data.index + 1)['1'].to_dict()
            return demand_dict
        except Exception as e:
            print(f"Error processing demand data: {str(e)}")
            return {}     

    def prepare_pyomo_data(self, num_periods=24, num_scenarios=5, num_tiers=4):
        """Prepare the data for the Pyomo model."""
        if self.gen_data is None:
            success = self.load_data()
            if not success:
                return None
                
        gen_data_dict = self.process_gen_data()
        storage_data_dict = self.process_storage_data()
        demand_data_dict = self.process_demand_data(num_periods)
        
        num_generators = len(gen_data_dict.get('CAP', {}))
        num_storage = len(storage_data_dict.get('E_MAX', {}))
        
        sets = {
            # 'T': {None: list(range(1, num_periods + 1))},
            'S': {None: list(range(1, num_scenarios + 1))},
            'G': {None: list(range(1, num_generators + 1))},
            'R': {None: list(range(1, num_tiers + 1))},
            'B': {None: list(range(1, num_storage + 1))}
        }
        
        reda_data = {}
        re_scenarios = {}
        
        fo_params = {
            # Manual values
            'D1': {None: 5},
            'D2': {None: 550.0},
            'PEN': {None: 2000},
            'PENDN': {None: 0},
            'smallM': {None: 0.01},
            # check probabilities
            'probTU': {r: 0.2 + 0.1 * (r-1) for r in range(1, num_tiers + 1)},
            'probTD': {r: 0.2 + 0.1 * (r-1) for r in range(1, num_tiers + 1)}
        }
        
        pyomo_data = {}
        pyomo_data.update(sets)
        pyomo_data.update(gen_data_dict)
        pyomo_data.update(storage_data_dict)
        pyomo_data.update({"DEMAND": demand_data_dict})
        pyomo_data.update(fo_params)
        
        # TODO - CALCULATE REDA, RE
        pyomo_data['REDA'] = reda_data
        pyomo_data['RE'] = re_scenarios
        pyomo_data = {None: pyomo_data}

        # TODO: UPDATE REQUIRED PARAMETERS
        required_params = [
            'CAP', 'VC', 'VCUP', 'VCDN', 'RR',
            'E_MAX', 'P_MAX', 'ETA_CH', 'ETA_DCH', 'E0', 'E_FINAL', 'STORAGE_COST',
            'D1', 'D2', 'PEN', 'PENDN', 'smallM', 'probTU', 'probTD',
            'DEMAND', 'REDA', 'RE'
        ]
        
        missing_params = [param for param in required_params if param not in pyomo_data[None]]
        if missing_params:
            print(f"Warning: Missing parameters: {missing_params}")
        
        return pyomo_data

In [49]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super().default(obj)

gen_csv_path = 'system_data/gen.csv'
storage_csv_path = 'system_data/storage.csv'
demand_csv_path = 'system_data/DAY_AHEAD_regional_load.csv'

system_data = SystemDataProcessor(gen_csv_path, storage_csv_path, demand_csv_path)
pyomo_system_data = system_data.prepare_pyomo_data()
print(json.dumps(pyomo_system_data[None], cls=NumpyEncoder, indent=4, sort_keys=True))

dafo_model = DAFOModel(
    num_periods=24,
    num_scenarios=1,
    num_generators=5,
    num_tiers=4,
    num_storage=5
)

da_instance = dafo_model.create_instance(pyomo_system_data)
solver_path = 'C:/Program Files/IBM/ILOG/CPLEX_Studio_Community2212/cplex/bin/x64_win64/cplex'
opt = pyo.SolverFactory('cplex',executable=solver_path)

opt.solve(da_instance)

Generator and Storage data loaded successfully. 5 generators and 5 storages found.
Demand data loaded successfully. 8784 periods found.
{
    "B": {
        "null": [
            1,
            2,
            3,
            4,
            5
        ]
    },
    "CAP": {
        "1": 20.0,
        "2": 20.0,
        "3": 76.0,
        "4": 76.0,
        "5": 20.0
    },
    "D1": {
        "null": 5
    },
    "D2": {
        "null": 550.0
    },
    "DEMAND": {
        "1": 985.0197922,
        "2": 985.7248887,
        "3": 1001.58956,
        "4": 1036.139287,
        "5": 1139.788471,
        "6": 1286.801089,
        "7": 1347.086838,
        "8": 1305.486145,
        "9": 1232.508659,
        "10": 1184.20955,
        "11": 1153.185304,
        "12": 1129.212024,
        "13": 1107.706581,
        "14": 1093.957199,
        "15": 1087.611331,
        "16": 1091.489362,
        "17": 1170.460168,
        "18": 1280.102672,
        "19": 1266.000742,
        "20": 1245.200396,
     

ValueError: Cannot load a SolverResults object with bad status: error

Model Solver

In [48]:
class ModelSolver:
    def __init__(self, solver_path=None):
        """Initialize the model solver with optional solver path"""
        self.solver_path = solver_path
        self.opt = self._setup_solver()
        
    def _setup_solver(self):
        """Setup the solver using CPLEX"""
        try:
            if self.solver_path:
                opt = pyo.SolverFactory('cplex', executable=self.solver_path)    
            else:
                opt = pyo.SolverFactory('cplex')
                    
            if not opt.available():
                print("ERROR: CPLEX is unavailable. Please check installation.")
                return None
               
            return opt
        except Exception as e:
            print(f"Error setting up solver: {str(e)}")
            return None
    
    def solve_da_model(self, da_data):
        """Solve the DA model with Flexibility Options"""
        try:
            # Create DAFOModel instance with explicit dimensions
            num_periods = 24  # Default
            num_scenarios = 5  # Default
            num_generators = 5  # Default
            num_tiers = 4  # Default
            num_storage = 3  # Default
            
            # Try to determine dimensions from the data
            if 'T' in da_data and None in da_data['T']:
                num_periods = len(da_data['T'][None])
            if 'S' in da_data and None in da_data['S']:
                num_scenarios = len(da_data['S'][None])
            if 'G' in da_data and None in da_data['G']:
                num_generators = len(da_data['G'][None])
            if 'R' in da_data and None in da_data['R']:
                num_tiers = len(da_data['R'][None])
            if 'B' in da_data and None in da_data['B']:
                num_storage = len(da_data['B'][None])
            
            # Validate that all required parameters exist
            required_params = ['DEMAND', 'REDA', 'RE', 'VC', 'VCUP', 'VCDN', 'CAP', 'RR',
                            'D1', 'D2', 'PEN', 'PENDN', 'smallM', 'probTU', 'probTD',
                            'E_MAX', 'P_MAX', 'ETA_CH', 'ETA_DCH', 'E0', 'E_FINAL', 'STORAGE_COST']
            
            # Print validation info
            print(f"Validating model parameters:")
            print(f"num_periods: {num_periods}")
            print(f"num_scenarios: {num_scenarios}")
            print(f"num_generators: {num_generators}")
            print(f"num_tiers: {num_tiers}")
            print(f"num_storage: {num_storage}")
            
            missing_params = [p for p in required_params if p not in da_data]
            if missing_params:
                print(f"WARNING: Missing parameters: {missing_params}")
                # Add default values for missing parameters
                for p in missing_params:
                    if p == 'DEMAND':
                        da_data[p] = {t: 100.0 for t in range(1, num_periods + 1)}
                    elif p == 'REDA':
                        da_data[p] = {t: 0.0 for t in range(1, num_periods + 1)}
                    elif p == 'RE':
                        da_data[p] = {(s, t): 0.0 for s in range(1, num_scenarios + 1) 
                                    for t in range(1, num_periods + 1)}
                    elif p in ['D1', 'D2', 'PEN', 'PENDN', 'smallM']:
                        da_data[p] = {None: 10.0 if p == 'D1' else 0.1 if p == 'D2' else 
                                    500.0 if p == 'PEN' else 200.0 if p == 'PENDN' else 0.0001}
                    elif p in ['probTU', 'probTD']:
                        da_data[p] = {r: 0.2 + 0.1 * (r-1) for r in range(1, num_tiers + 1)}
                    # Add other parameter defaults as needed
            
            # Validate indices in DEMAND parameter
            if 'DEMAND' in da_data:
                missing_times = [t for t in range(1, num_periods + 1) if t not in da_data['DEMAND']]
                if missing_times:
                    print(f"WARNING: Missing time periods in DEMAND: {missing_times}")
                    for t in missing_times:
                        da_data['DEMAND'][t] = 100.0  # Default value
            
            dafo_model = DAFOModel(
                num_periods=num_periods,
                num_scenarios=num_scenarios,
                num_generators=num_generators,
                num_tiers=num_tiers,
                num_storage=num_storage
            )
            
            # Debug: print a subset of the DEMAND parameter
            print(f"Sample of DEMAND parameter: {dict(list(da_data['DEMAND'].items())[:5])}")
            
            # Create model instance
            instance = dafo_model.create_instance(da_data)
            print("DA model instance created successfully")
            
            # Solve model
            result = self.solve_with_handling(instance)
            
            if result:
                # Store key outputs
                outputs = self._extract_da_outputs(instance)
                return instance, outputs
            else:
                print("Failed to solve DA model")
                return None, None
                
        except Exception as e:
            print(f"Error in DA model: {str(e)}")
            import traceback
            traceback.print_exc()
            return None, None
    
    def solve_rt_model(self, rt_data, da_outputs=None):
        """Solve the RT model for each scenario"""
        try:
            # Create RTSimModel instance with explicit dimensions
            num_periods = len(rt_data.get('T', {}).get(None, [24]))
            num_scenarios = len(rt_data.get('S', {}).get(None, [5]))
            num_generators = len(rt_data.get('G', {}).get(None, [5]))
            num_storage = len(rt_data.get('B', {}).get(None, [3]))
            
            rtsim_model = RTSimModel(
                num_periods=num_periods,
                num_scenarios=num_scenarios,
                num_generators=num_generators,
                num_storage=num_storage
            )
            
            # Add DA outputs to RT data if provided
            if da_outputs:
                for key, value in da_outputs.items():
                    if key not in rt_data:
                        rt_data[key] = value
            
            # Create model instance
            instance = rtsim_model.create_instance(rt_data)
            print("RT model instance created successfully")
            
            # Solve model
            result = self.solve_with_handling(instance)
            
            if result:
                # Store key outputs
                outputs = self._extract_rt_outputs(instance)
                return instance, outputs
            else:
                print("Failed to solve RT model")
                return None, None
                
        except Exception as e:
            print(f"Error in RT model: {str(e)}")
            return None, None
    
    def solve_with_handling(self, model):
        """Solve a Pyomo model with error handling"""
        try:
            result = self.opt.solve(model, tee=True)
            
            if (result.solver.status == pyo.SolverStatus.ok and 
                result.solver.termination_condition == pyo.TerminationCondition.optimal):
                print("Model solved successfully")
                return True
            else:
                print(f"Solver status: {result.solver.status}")
                print(f"Termination condition: {result.solver.termination_condition}")
                return False
                
        except Exception as e:
            print(f"Error solving model: {str(e)}")
            return False
    
    def _extract_da_outputs(self, instance):
        """Extract key outputs from DA model instance"""
        outputs = {}
        
        try:
            # Extract energy schedules
            if hasattr(instance, 'xDA'):
                outputs['xDA'] = {(g, t): instance.xDA[g, t].value for g in instance.G for t in instance.T}
            
            # Extract renewable schedule
            if hasattr(instance, 'rgDA'):
                outputs['REDA'] = {t: instance.rgDA[t].value for t in instance.T}
            
            # Extract storage schedules if storage exists
            if hasattr(instance, 'e') and hasattr(instance, 'p_ch') and hasattr(instance, 'p_dch'):
                outputs['e_DA'] = {(b, t): instance.e[b, t].value for b in instance.B for t in instance.T}
                outputs['p_ch_DA'] = {(b, t): instance.p_ch[b, t].value for b in instance.B for t in instance.T}
                outputs['p_dch_DA'] = {(b, t): instance.p_dch[b, t].value for b in instance.B for t in instance.T}
            
            # Extract FO schedules for all suppliers (generators + storage)
            if hasattr(instance, 'hsu') and hasattr(instance, 'hsd'):
                outputs['hsu'] = {(r, g, t): instance.hsu[r, g, t].value for r in instance.R for g in instance.G for t in instance.T}
                outputs['hsd'] = {(r, g, t): instance.hsd[r, g, t].value for r in instance.R for g in instance.G for t in instance.T}
            
            if hasattr(instance, 'bsu') and hasattr(instance, 'bsd'):
                outputs['bsu'] = {(r, b, t): instance.bsu[r, b, t].value for r in instance.R for b in instance.B for t in instance.T}
                outputs['bsd'] = {(r, b, t): instance.bsd[r, b, t].value for r in instance.R for b in instance.B for t in instance.T}
            
            # Extract FO demand
            if hasattr(instance, 'hdu') and hasattr(instance, 'hdd'):
                outputs['hdu'] = {(r, t): instance.hdu[r, t].value for r in instance.R for t in instance.T}
                outputs['hdd'] = {(r, t): instance.hdd[r, t].value for r in instance.R for t in instance.T}
            
            # Extract prices
            if hasattr(instance, 'dual'):
                if hasattr(instance, 'Con3'):
                    outputs['DA_price'] = {t: instance.dual[instance.Con3[t]] for t in instance.T}
                if hasattr(instance, 'Con4UP'):
                    outputs['FO_up_price'] = {(r, t): instance.dual[instance.Con4UP[r, t]] for r in instance.R for t in instance.T}
                if hasattr(instance, 'Con4DN'):
                    outputs['FO_down_price'] = {(r, t): instance.dual[instance.Con4DN[r, t]] for r in instance.R for t in instance.T}
            
            # Extract total costs
            if hasattr(instance, 'OBJ'):
                outputs['total_cost'] = pyo.value(instance.OBJ)
            
            # Extract demand slack
            if hasattr(instance, 'd'):
                outputs['DAdr'] = {t: instance.d[t].value for t in instance.T}
            
        except Exception as e:
            print(f"Error extracting DA outputs: {str(e)}")
        
        return outputs
    
    def _extract_rt_outputs(self, instance):
        """Extract key outputs from RT model instance"""
        outputs = {}
        
        try:
            # Extract RT energy adjustments if generators exist
            if hasattr(instance, 'xup') and hasattr(instance, 'xdn'):
                outputs['xup'] = {(s, g, t): instance.xup[s, g, t].value for s in instance.S for g in instance.G for t in instance.T}
                outputs['xdn'] = {(s, g, t): instance.xdn[s, g, t].value for s in instance.S for g in instance.G for t in instance.T}
            
            # Extract storage adjustments if storage exists
            if hasattr(instance, 'e') and hasattr(instance, 'p_ch') and hasattr(instance, 'p_dch'):
                outputs['e_RT'] = {(s, b, t): instance.e[s, b, t].value for s in instance.S for b in instance.B for t in instance.T}
                outputs['p_ch_RT'] = {(s, b, t): instance.p_ch[s, b, t].value for s in instance.S for b in instance.B for t in instance.T}
                outputs['p_dch_RT'] = {(s, b, t): instance.p_dch[s, b, t].value for s in instance.S for b in instance.B for t in instance.T}
            
            if hasattr(instance, 'b_up') and hasattr(instance, 'b_dn'):
                outputs['b_up'] = {(s, b, t): instance.b_up[s, b, t].value for s in instance.S for b in instance.B for t in instance.T}
                outputs['b_dn'] = {(s, b, t): instance.b_dn[s, b, t].value for s in instance.S for b in instance.B for t in instance.T}
            
            # Extract RE adjustments
            if hasattr(instance, 'rgup') and hasattr(instance, 'rgdn'):
                outputs['rgup'] = {(s, t): instance.rgup[s, t].value for s in instance.S for t in instance.T}
                outputs['rgdn'] = {(s, t): instance.rgdn[s, t].value for s in instance.S for t in instance.T}
            
            # Extract shortage/surplus
            if hasattr(instance, 'sdup') and hasattr(instance, 'sddn'):
                outputs['sdup'] = {(s, t): instance.sdup[s, t].value for s in instance.S for t in instance.T}
                outputs['sddn'] = {(s, t): instance.sddn[s, t].value for s in instance.S for t in instance.T}
            
            # Extract RT prices
            if hasattr(instance, 'dual') and hasattr(instance, 'Con3'):
                outputs['RT_price'] = {(s, t): instance.dual[instance.Con3[s, t]] for s in instance.S for t in instance.T}
            
            # Extract total costs per scenario
            if hasattr(instance, 'prob') and hasattr(instance, 'VCUP') and hasattr(instance, 'VCDN'):
                outputs['scenario_costs'] = {}
                for s in instance.S:
                    scenario_cost = 0
                    if hasattr(instance, 'xup') and hasattr(instance, 'xdn'):
                        scenario_cost += sum(
                            instance.prob[s] * (
                                sum(instance.VCUP[g] * instance.xup[s, g, t].value - 
                                    instance.VCDN[g] * instance.xdn[s, g, t].value 
                                    for g in instance.G for t in instance.T)
                            ))
                    outputs['scenario_costs'][s] = scenario_cost
            
        except Exception as e:
            print(f"Error extracting RT outputs: {str(e)}")
        
        return outputs

In [49]:
def main():
    input_file = 'sample.csv'
    output_dir = 'Flexibility_Options_Results'
    solver_path = 'C:/Program Files/IBM/ILOG/CPLEX_Studio_Community2212/cplex/bin/x64_win64/cplex'

    os.makedirs(output_dir, exist_ok=True)
    
    try:
        print("Starting Flexibility Options with Storage Analysis...")
        
        # Initialize data processor with explicit dimensions
        processor = DataProcessor(input_file)
        print("Data processor initialized.")
        
        # Prepare DA and RT data
        da_data = processor.prepare_da_data()
        rt_data = processor.prepare_rt_data()
        print("Data preparation completed.")
        
        # Initialize model solver
        solver = ModelSolver(solver_path)
        if solver.opt is None:
            print("ERROR: Solver initialization failed. Exiting...")
            return
        print("Model solver initialized.")
        
        # Solve DA model
        print("\nSolving DA model with Flexibility Options...")
        da_instance, da_outputs = solver.solve_da_model(da_data)
        
        if da_instance is None or da_outputs is None:
            print("ERROR: DA model solution failed. Exiting...")
            return
        print("DA model solved successfully.")
        
        # Export DA model solution to CSV
        export_solution_to_csv(da_outputs, os.path.join(output_dir, 'da_solution.csv'))
        
        # Solve RT model
        print("\nSolving RT model for each scenario...")
        # Update RT data with DA outputs
        for key, value in da_outputs.items():
            if key in ['xDA', 'REDA', 'e_DA', 'p_ch_DA', 'p_dch_DA', 'DAdr']:
                rt_data[key] = value
        
        rt_instance, rt_outputs = solver.solve_rt_model(rt_data)
        
        if rt_instance is None or rt_outputs is None:
            print("ERROR: RT model solution failed. Exiting...")
            return
        print("RT model solved successfully.")
        
        # Export RT model solution to CSV
        export_solution_to_csv(rt_outputs, os.path.join(output_dir, 'rt_solution.csv'))
        
        # Analyze results
        print("\nAnalyzing results...")
        analyzer = ResultsAnalyzer(da_outputs, rt_outputs)
        analysis_results = analyzer.run_full_analysis()
        print("Analysis completed and visualizations generated.")
        
        # Export key model outputs for further analysis
        export_model_outputs(da_instance, rt_instance, os.path.join(output_dir, 'model_outputs.xlsx'))
        print(f"Model outputs exported to {os.path.join(output_dir, 'model_outputs.xlsx')}")
        
        print("\nSummary:")
        print(f"- Total DA cost: ${da_outputs.get('total_cost', 0):.2f}")
        
        if analysis_results and 'fo_summary' in analysis_results and 'storage_fo_contribution' in analysis_results['fo_summary']:
            storage_contrib = analysis_results['fo_summary']['storage_fo_contribution']
            avg_storage_contrib = np.mean([v for k, v in storage_contrib.items()]) * 100
            print(f"- Average storage contribution to FOs: {avg_storage_contrib:.2f}%")
        
        print(f"- Full results available in: {output_dir}")
        
    except Exception as e:
        print(f"ERROR: {str(e)}")

def export_solution_to_csv(outputs, filename):
    """Export model solution to CSV file"""
    try:
        # Convert nested dictionaries to DataFrame
        data = []
        for key, values in outputs.items():
            if isinstance(values, dict):
                for idx, val in values.items():
                    if isinstance(idx, tuple):
                        # Handle multi-index keys
                        row = list(idx)
                        row_dict = {'parameter': key, 'value': val}
                        for i, dim in enumerate(idx):
                            row_dict[f'dim{i+1}'] = dim
                        data.append(row_dict)
                    else:
                        # Handle single-index keys
                        data.append({'parameter': key, 'dim1': idx, 'value': val})
            else:
                # Handle scalar values
                data.append({'parameter': key, 'value': values})
        
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)
        print(f"Solution exported to {filename}")
    except Exception as e:
        print(f"Error exporting solution: {str(e)}")

def export_model_outputs(da_instance, rt_instance, filename):
    """Export raw model outputs for further analysis"""
    try:
        with pd.ExcelWriter(filename) as writer:
            # Export dual variables from DA model
            da_duals = {}
            for c in da_instance.component_objects(pyo.Constraint, active=True):
                for idx in c:
                    if idx in da_instance.dual:
                        da_duals[f"{c.name}_{idx}"] = da_instance.dual[idx]
            
            pd.Series(da_duals).to_frame('Value').to_excel(writer, 'DA_Duals')
            
            # Export storage-related variables from DA model
            storage_vars = {}
            for v_name in ['e', 'p_ch', 'p_dch', 'bsu', 'bsd']:
                if hasattr(da_instance, v_name):
                    v = getattr(da_instance, v_name)
                    for idx in v:
                        storage_vars[f"{v_name}_{idx}"] = v[idx].value
            
            pd.Series(storage_vars).to_frame('Value').to_excel(writer, 'DA_Storage_Variables')
            
            # Export RT energy prices
            if hasattr(rt_instance, 'dual'):
                rt_prices = {}
                for s in rt_instance.S:
                    for t in rt_instance.T:
                        if hasattr(rt_instance, 'Con3') and (s, t) in rt_instance.Con3:
                            if (s, t) in rt_instance.dual:
                                rt_prices[f"RT_price_s{s}_t{t}"] = rt_instance.dual[(s, t)]
                
                pd.Series(rt_prices).to_frame('Value').to_excel(writer, 'RT_Prices')
                
    except Exception as e:
        print(f"Error exporting model outputs: {str(e)}")

if __name__ == "__main__":
    main()

Starting Flexibility Options with Storage Analysis...
Data processor initialized.
DA model parameters:
VC: {1: 20.0, 2: 30.0, 3: 40.0, 4: 50.0, 5: 60.0}
VCUP: {1: 30.0, 2: 45.0, 3: 60.0, 4: 75.0, 5: 90.0}
VCDN: {1: 10.0, 2: 15.0, 3: 20.0, 4: 25.0, 5: 30.0}
CAP: {1: 100.0, 2: 20.0, 3: 20.0, 4: 20.0, 5: 20.0}
RR: {1: 20.0, 2: 10.0, 3: 10.0, 4: 10.0, 5: 10.0}
RT model parameters:
VC: {1: 20.0, 2: 30.0, 3: 40.0, 4: 50.0, 5: 60.0}
VCUP: {1: 30.0, 2: 45.0, 3: 60.0, 4: 75.0, 5: 90.0}
VCDN: {1: 10.0, 2: 15.0, 3: 20.0, 4: 25.0, 5: 30.0}
CAP: {1: 100.0, 2: 20.0, 3: 20.0, 4: 20.0, 5: 20.0}
RR: {1: 20.0, 2: 10.0, 3: 10.0, 4: 10.0, 5: 10.0}
Data preparation completed.
Model solver initialized.

Solving DA model with Flexibility Options...
Validating model parameters:
num_periods: 24
num_scenarios: 5
num_generators: 5
num_tiers: 4
num_storage: 3
Sample of DEMAND parameter: {1: 120.0, 2: 125.0, 3: 135.0, 4: 150.0, 5: 160.0}
ERROR: Rule failed when generating expression for Constraint Con3 with index


Traceback (most recent call last):
  File "C:\Users\hanbo\AppData\Local\Temp\ipykernel_7428\853726351.py", line 98, in solve_da_model
    instance = dafo_model.create_instance(da_data)
  File "C:\Users\hanbo\AppData\Local\Temp\ipykernel_7428\3517868776.py", line 261, in create_instance
    return self.model.create_instance(data)
           ~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "c:\Users\hanbo\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyomo\core\base\PyomoModel.py", line 734, in create_instance
    instance.load(data, namespaces=_namespaces, profile_memory=profile_memory)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hanbo\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyomo\core\base\PyomoModel.py", line 771, in load
    self._load_model_data(dp, namespaces, profile_memory=profile_memory)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hanbo\AppData\Local\Programs\Py

In [50]:
import matplotlib.pyplot as plt

def plot_summary_results(summary_file):
    try:
        # Read the Excel file
        system_cost = pd.read_excel(summary_file, sheet_name='System_Cost', index_col=0)
        price_conv = pd.read_excel(summary_file, sheet_name='Price_Convergence', index_col=0)
        premium_up = pd.read_excel(summary_file, sheet_name='Premium_Up', index_col=0)
        premium_down = pd.read_excel(summary_file, sheet_name='Premium_Down', index_col=0)
        demand_cost = pd.read_excel(summary_file, sheet_name='Demand_Cost', index_col=0)
        curtail_cost = pd.read_excel(summary_file, sheet_name='Curtailment_Cost', index_col=0)
        
        # Check if dataframes contain numeric data
        if (system_cost.empty or price_conv.empty or premium_up.empty or 
            premium_down.empty or demand_cost.empty or curtail_cost.empty):
            print("Error: One or more sheets contain no data")
            return
            
        # Convert data to numeric, replacing non-numeric values with NaN
        system_cost = system_cost.apply(pd.to_numeric, errors='coerce')
        price_conv = price_conv.apply(pd.to_numeric, errors='coerce')
        premium_up = premium_up.apply(pd.to_numeric, errors='coerce')
        premium_down = premium_down.apply(pd.to_numeric, errors='coerce')
        demand_cost = demand_cost.apply(pd.to_numeric, errors='coerce')
        curtail_cost = curtail_cost.apply(pd.to_numeric, errors='coerce')
        
        # Create subplots
        fig, axes = plt.subplots(3, 2, figsize=(15, 15))
        fig.suptitle('Summary Results Visualization')
        
        # Plot system cost
        if not system_cost.empty and system_cost.notna().any().any():
            system_cost.plot(kind='bar', ax=axes[0,0], title='System Cost')
            axes[0,0].set_ylabel('Cost')
            axes[0,0].tick_params(axis='x', rotation=45)
        else:
            axes[0,0].set_title('System Cost - No numeric data available')
        
        # Plot price convergence
        if not price_conv.empty and price_conv.notna().any().any():
            price_conv.plot(kind='bar', ax=axes[0,1], title='Price Convergence')
            axes[0,1].set_ylabel('Price')
            axes[0,1].tick_params(axis='x', rotation=45)
        else:
            axes[0,1].set_title('Price Convergence - No numeric data available')
        
        # Plot premium up/down
        if not premium_up.empty and premium_up.notna().any().any():
            premium_up.plot(kind='bar', ax=axes[1,0], title='Premium Up')
            axes[1,0].set_ylabel('Premium')
            axes[1,0].tick_params(axis='x', rotation=45)
        else:
            axes[1,0].set_title('Premium Up - No numeric data available')
        
        if not premium_down.empty and premium_down.notna().any().any():
            premium_down.plot(kind='bar', ax=axes[1,1], title='Premium Down')
            axes[1,1].set_ylabel('Premium')
            axes[1,1].tick_params(axis='x', rotation=45)
        else:
            axes[1,1].set_title('Premium Down - No numeric data available')
        
        # Plot demand and curtailment costs
        if not demand_cost.empty and demand_cost.notna().any().any():
            demand_cost.plot(kind='bar', ax=axes[2,0], title='Demand Cost')
            axes[2,0].set_ylabel('Cost')
            axes[2,0].tick_params(axis='x', rotation=45)
        else:
            axes[2,0].set_title('Demand Cost - No numeric data available')
        
        if not curtail_cost.empty and curtail_cost.notna().any().any():
            curtail_cost.plot(kind='bar', ax=axes[2,1], title='Curtailment Cost')
            axes[2,1].set_ylabel('Cost')
            axes[2,1].tick_params(axis='x', rotation=45)
        else:
            axes[2,1].set_title('Curtailment Cost - No numeric data available')
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error creating visualizations: {str(e)}")

# Example usage for visualization
if __name__ == "__main__":
    summary_file = os.path.join('Test_output_files/Results_SectionV', f"Summary_Results_{date.today().strftime('%Y_%m_%d')}.xlsx")
    plot_summary_results(summary_file)


Error creating visualizations: [Errno 2] No such file or directory: 'Test_output_files/Results_SectionV\\Summary_Results_2025_02_27.xlsx'
